In [3]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/question-answer/100_Unique_QA_Dataset.csv


In [4]:
import pandas as pd

df = pd.read_csv('/kaggle/input/question-answer/100_Unique_QA_Dataset.csv')

df.head()

,question,answer
0,What is the capital of France?,Paris
1,What is the capital of Germany?,Berlin
2,Who wrote 'To Kill a Mockingbird'?,Harper-Lee
3,What is the largest planet in our solar system?,Jupiter
4,What is the boiling point of water in Celsius?,100


In [5]:
# tokenize
def tokenize(text):
  text = text.lower()
  text = text.replace('?','')
  text = text.replace("'","")
  return text.split()

In [6]:
tokenize('What is the capital of France?')

['what', 'is', 'the', 'capital', 'of', 'france']

In [7]:
# vocab
vocab = {'<UNK>':0}

In [8]:
def build_vocab(row):
  tokenized_question = tokenize(row['question'])
  tokenized_answer = tokenize(row['answer'])

  merged_tokens = tokenized_question + tokenized_answer

  for token in merged_tokens:

    if token not in vocab:
      vocab[token] = len(vocab)

In [9]:
df.apply(build_vocab, axis=1)

0     None
1     None
2     None
3     None
4     None
      ... 
85    None
86    None
87    None
88    None
89    None
Length: 90, dtype: object

In [30]:
list(vocab.keys())

['<UNK>',
 'what',
 'is',
 'the',
 'capital',
 'of',
 'france',
 'paris',
 'germany',
 'berlin',
 'who',
 'wrote',
 'to',
 'kill',
 'a',
 'mockingbird',
 'harper-lee',
 'largest',
 'planet',
 'in',
 'our',
 'solar',
 'system',
 'jupiter',
 'boiling',
 'point',
 'water',
 'celsius',
 '100',
 'painted',
 'mona',
 'lisa',
 'leonardo-da-vinci',
 'square',
 'root',
 '64',
 '8',
 'chemical',
 'symbol',
 'for',
 'gold',
 'au',
 'which',
 'year',
 'did',
 'world',
 'war',
 'ii',
 'end',
 '1945',
 'longest',
 'river',
 'nile',
 'japan',
 'tokyo',
 'developed',
 'theory',
 'relativity',
 'albert-einstein',
 'freezing',
 'fahrenheit',
 '32',
 'known',
 'as',
 'red',
 'mars',
 'author',
 '1984',
 'george-orwell',
 'currency',
 'united',
 'kingdom',
 'pound',
 'india',
 'delhi',
 'discovered',
 'gravity',
 'newton',
 'how',
 'many',
 'continents',
 'are',
 'there',
 'on',
 'earth',
 '7',
 'gas',
 'do',
 'plants',
 'use',
 'photosynthesis',
 'co2',
 'smallest',
 'prime',
 'number',
 '2',
 'invented'

In [10]:
len(vocab)

324

In [11]:
# convert words to numerical indices
def text_to_indices(text, vocab):

  indexed_text = []

  for token in tokenize(text):

    if token in vocab:
      indexed_text.append(vocab[token])
    else:
      indexed_text.append(vocab['<UNK>'])

  return indexed_text

In [13]:
text_to_indices("What is dinasour <UNK>", vocab)

[1, 2, 0, 0]

In [14]:
import torch
from torch.utils.data import Dataset, DataLoader

In [15]:
class QADataset(Dataset):

  def __init__(self, df, vocab):
    self.df = df
    self.vocab = vocab

  def __len__(self):
    return self.df.shape[0]

  def __getitem__(self, index):

    numerical_question = text_to_indices(self.df.iloc[index]['question'], self.vocab)
    numerical_answer = text_to_indices(self.df.iloc[index]['answer'], self.vocab)

    return torch.tensor(numerical_question), torch.tensor(numerical_answer)

In [16]:
dataset = QADataset(df, vocab)

In [17]:
dataloader = DataLoader(dataset, batch_size=1, shuffle=True)

In [18]:
for question, answer in dataloader:
  print(question, answer[0])

tensor([[ 42, 125,   2,  62,  63,   3, 126, 127]]) tensor([128])
tensor([[  1,   2,   3,  92, 137,  19,   3,  45]]) tensor([185])
tensor([[ 1,  2,  3, 50, 51, 19,  3, 45]]) tensor([52])
tensor([[  1,   2,   3, 163, 164, 165,  83,  84]]) tensor([166])
tensor([[ 42, 117, 118,   3, 119,  94, 120]]) tensor([121])
tensor([[  1,   2,   3, 234,   5, 235]]) tensor([131])
tensor([[ 42,  18,   2,   3, 281,  12,   3, 282]]) tensor([205])
tensor([[  1,   2,   3,   4,   5, 113]]) tensor([114])
tensor([[ 42, 312,   2, 313,  62,  63,   3, 314, 315]]) tensor([316])
tensor([[10,  2,  3, 66,  5, 67]]) tensor([68])
tensor([[ 10,  11, 189, 158, 190]]) tensor([191])
tensor([[ 10,  75, 111]]) tensor([112])
tensor([[  1,   2,   3, 212,   5,  14, 213, 214]]) tensor([215])
tensor([[  1,   2,   3, 221,   5, 222, 223, 224]]) tensor([225])
tensor([[ 42,  18, 118,   3, 186, 187]]) tensor([188])
tensor([[  1,   2,   3, 146, 147,  19, 148]]) tensor([149])
tensor([[10, 75, 76]]) tensor([77])
tensor([[ 42, 290, 291, 1

In [19]:
import torch.nn as nn

In [20]:
class SimpleRNN(nn.Module):

  def __init__(self, vocab_size):
    super().__init__()
    self.embedding = nn.Embedding(vocab_size, embedding_dim=50)
    self.rnn = nn.RNN(50, 64, batch_first=True)
    self.fc = nn.Linear(64, vocab_size)

  def forward(self, question):
    embedded_question = self.embedding(question)
    hidden, final = self.rnn(embedded_question)
    output = self.fc(final.squeeze(0))

    return output

In [21]:
x = nn.Embedding(324, embedding_dim=50)
y = nn.RNN(50, 64, batch_first=True)
z = nn.Linear(64, 324)

a = dataset[0][0].reshape(1,6)
print("shape of a:", a.shape)
b = x(a)
print("shape of b:", b.shape)
c, d = y(b)
print("shape of c:", c.shape)
print("shape of d:", d.shape)

e = z(d.squeeze(0))

print("shape of e:", e.shape)

shape of a: torch.Size([1, 6])
shape of b: torch.Size([1, 6, 50])
shape of c: torch.Size([1, 6, 64])
shape of d: torch.Size([1, 1, 64])
shape of e: torch.Size([1, 324])


In [22]:
learning_rate = 0.001
epochs = 20

In [23]:
model = SimpleRNN(len(vocab))

In [24]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [25]:
# training loop

for epoch in range(epochs):

  total_loss = 0

  for question, answer in dataloader:

    optimizer.zero_grad()

    # forward pass
    output = model(question)

    # loss -> output shape (1,324) - (1)
    loss = criterion(output, answer[0])

    # gradients
    loss.backward()

    # update
    optimizer.step()

    total_loss = total_loss + loss.item()

  print(f"Epoch: {epoch+1}, Loss: {total_loss:4f}")

Epoch: 1, Loss: 525.126700
Epoch: 2, Loss: 457.185567
Epoch: 3, Loss: 378.507101
Epoch: 4, Loss: 314.964952
Epoch: 5, Loss: 262.697199
Epoch: 6, Loss: 213.545950
Epoch: 7, Loss: 168.741948
Epoch: 8, Loss: 131.532205
Epoch: 9, Loss: 101.187878
Epoch: 10, Loss: 76.676179
Epoch: 11, Loss: 59.228433
Epoch: 12, Loss: 46.522539
Epoch: 13, Loss: 36.934703
Epoch: 14, Loss: 30.296675
Epoch: 15, Loss: 24.976362
Epoch: 16, Loss: 21.045876
Epoch: 17, Loss: 17.717399
Epoch: 18, Loss: 15.137509
Epoch: 19, Loss: 13.007034
Epoch: 20, Loss: 11.347425


In [36]:
def predict(model, question, threshold=0.5):

  # convert question to numbers
  numerical_question = text_to_indices(question, vocab)

  # tensor
  question_tensor = torch.tensor(numerical_question).unsqueeze(0)

  # send to model
  output = model(question_tensor)

  # convert logits to probs
  probs = torch.nn.functional.softmax(output, dim=1)

  # find index of max prob
  value, index = torch.max(probs, dim=1)

  if value < threshold:
    print("I don't know")
  else:
    print(list(vocab.keys())[index])

In [27]:
predict(model, "What is the largest planet in our solar system?")

jupiter


In [31]:
predict(model, "What is approximately largest planet in our solar system?")

jupiter


In [32]:
predict(model,'what is capital of india')

delhi


In [37]:
predict(model,'what is capital of sri lanka')

I don't know
